# Creating new tables

In [1]:
import sqlite3
import pandas as pd

# connect to database
# HERE I USE FAKE DATABASE WITH RANDOM SENSELESS DATA
con = sqlite3.connect("university_professors.db")
cur = con.cursor()

In [2]:
cur.execute("""CREATE TABLE IF NOT EXISTS professors (
                firstname text,
                lastname text
               );"""
               )

In [3]:
cur.execute("""CREATE TABLE IF NOT EXISTS universities (
                university_shortname text,
                university text,
                university_city text
               );"""
               )

In [4]:
pd.read_sql("""SELECT * 
               FROM universities
               ;""", 
            con)

,university_shortname,university,university_city


## Adding columns to existing tables is easy, especially if they're still empty.

To add columns you can use the following SQL query:

```
ALTER TABLE table_name
ADD COLUMN column_name data_type;
```

In [134]:
cur.execute("""ALTER TABLE professors
               ADD COLUMN university_shortname text
               ;"""
               )

In [ ]:
# Insert unique professors into the new table
pd.read_sql("""INSERT INTO professors 
               SELECT firstname, lastname, university_shortname 
               FROM university_professors
               ;""", 
            con)

In [7]:
pd.read_sql("""SELECT * 
               FROM professors
               ;""", 
            con)

,firstname,lastname,university_shortname


In [8]:
cur.execute("""CREATE TABLE affiliations (
                firstname text,
                lastname text,
                university_shortname text,
                function text,
                organisation text
               );"""
               )

- ### To rename columns:
```
ALTER TABLE table_name
RENAME COLUMN old_name TO new_name;
```

- ### To delete columns:
```
ALTER TABLE table_name
DROP COLUMN column_name;
```

In [9]:
cur.execute("""ALTER TABLE affiliations
               RENAME COLUMN organisation TO organization
               ;"""
               )

In [10]:
cur.execute("""ALTER TABLE affiliations
               DROP COLUMN university_shortname
               ;"""
               )

- ### For table deletion, you can use the simple command:

```
DROP TABLE table_name;
```

- ### The syntax for changing the data type of a column

```
ALTER TABLE table_name
ALTER COLUMN column_name
TYPE varchar(10)
```

In [ ]:
cur.execute("""ALTER TABLE professors
               ALTER COLUMN university_shortname
               TYPE char(3)
               ;"""
               )

In [ ]:
cur.execute("""ALTER TABLE professors 
               ALTER COLUMN firstname 
               TYPE varchar(10)
               USING SUBSTRING(firstname FROM 1 FOR 10)
               ;"""
               )

In [5]:
# Create table organizations
cur.execute("""CREATE TABLE IF NOT EXISTS organizations (
                organization varchar(64),
                organization_sector varchar(64)
               );"""
               )

# Key constraints

- ### Syntax for adding primary key to the existing table:
```
ALTER TABLE table_name
ADD CONSTRAINT some_name PRIMARY KEY (column_name)
```

In [10]:
# Rename the organization column to id
cur.execute("""ALTER TABLE organizations
               RENAME COLUMN organization TO id
               ;"""
               )

In [ ]:
# Make id a primary key
cur.execute("""ALTER TABLE organizations
               ADD CONSTRAINT organization_pk PRIMARY KEY (id)
               ;"""
               )

- ### A special data type *serial* turns the column into an auto-incrementing number

In [12]:
# Add the new column to the table
cur.execute("""ALTER TABLE professors 
               ADD COLUMN id serial
               ;"""
               )

In [ ]:
# Make id a primary key
cur.execute("""ALTER TABLE professors 
               ADD CONSTRAINT professors_pkey PRIMARY KEY(id)
               ;"""
               )

# Foreign keys

- ### Syntax for adding foreign key to the existing table:
```
ALTER TABLE a 
ADD CONSTRAINT a_fkey FOREIGN KEY (b_id) REFERENCES b (id);
```

In [ ]:
# example in pure SQL

# Rename the university_shortname column
ALTER TABLE professors
RENAME COLUMN university_shortname TO university_id;

# Add a foreign key on professors referencing universities
ALTER TABLE professors 
ADD CONSTRAINT professors_fkey FOREIGN KEY (university_id) REFERENCES universities (id);

- ### Here's a way to update columns of a table based on values in another table:

```
UPDATE table_a
SET column_to_update = table_b.column_to_update_from
FROM table_b
WHERE condition1 AND condition2 AND ...;
```

# Referential integrity
- ### enforced through foreign keys

### Altering a foreign key constraint behavior doesn't work with ALTER COLUMN. Instead, you have to DROP the key constraint and then ADD a new one with a different ON DELETE behavior.

### For deleting constraints, though, you need to know their name. This information is also stored in information_schema.

In [ ]:
# Identify the correct constraint name

pd.read_sql("""SELECT constraint_name, table_name, constraint_type
               FROM information_schema.table_constraints
               WHERE constraint_type = 'FOREIGN KEY'
               ;""", 
            con)

### In database design, you have to strike a balance between modeling overhead, desired data consistency, and usability for queries. Congratulations, you made it to the end!